In [5]:
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras import Input
from scipy.optimize import least_squares
import tensorflow as tf

# Set random seed for reproducibility
tf.random.set_seed(42)
np.random.seed(42)

# Load the dataset
from google.colab import drive
drive.mount('/content/drive')
file_path = '/content/drive/MyDrive/withouticorvaafterwrapper0256.xlsx'
data = pd.read_excel(file_path)

# Define columns based on types
binary_features = ['Gendermale1', 'BloodPressure']
numeric_features = ['QualityofSleep', 'HeartRate']

target = 'SleepDisorder'

# Define ColumnTransformer for preprocessing
preprocessor = ColumnTransformer(
    transformers=[
        ('binary', MinMaxScaler(), binary_features),
        ('numeric', StandardScaler(), numeric_features),
    ]
)

# Separate features (X) and target variable (y)
X = data.drop(columns=[target])
y = data[target]

# Apply preprocessing to the entire dataset
X = preprocessor.fit_transform(X)

def build_model(input_dim):
    model = Sequential([
        Input(shape=(input_dim,)),
        Dense(10, activation='sigmoid'),
        Dense(3, activation='sigmoid'),
        Dense(1, activation='sigmoid')
    ])
    return model

def loss_function(weights, X, y, model):
    offset = 0
    reshaped_weights = []
    for layer_weights in model.get_weights():
        layer_shape = layer_weights.shape
        layer_size = np.prod(layer_shape)
        reshaped_weights.append(weights[offset:offset + layer_size].reshape(layer_shape))
        offset += layer_size

    model.set_weights(reshaped_weights)
    predictions = model.predict(X, verbose=0).flatten()
    residuals = y - predictions
    return residuals

def optimize_levenberg_marquardt(X_train, y_train, model):
    initial_weights = np.concatenate([w.flatten() for w in model.get_weights()])

    epoch_accuracies = []
    epoch_losses = []

    def reshape_weights(weights):
        offset = 0
        reshaped_weights = []
        for layer_weights in model.get_weights():
            layer_shape = layer_weights.shape
            layer_size = np.prod(layer_shape)
            reshaped_weights.append(weights[offset:offset + layer_size].reshape(layer_shape))
            offset += layer_size
        return reshaped_weights

    def fun_with_logging(weights):
        reshaped_weights = reshape_weights(weights)
        model.set_weights(reshaped_weights)
        predictions = model.predict(X_train, verbose=0).flatten()
        accuracy = np.mean((predictions.round() == y_train))
        loss = np.mean((y_train - predictions) ** 2)
        epoch_accuracies.append(accuracy)
        epoch_losses.append(loss)
        return loss_function(weights, X_train, y_train, model)

    result = least_squares(
        fun=fun_with_logging,
        x0=initial_weights,
        method='lm',
        verbose=2
    )

    model.set_weights(reshape_weights(result.x))
    return model, epoch_accuracies, epoch_losses

def calculate_metrics(y_true, y_pred):
    y_pred_rounded = y_pred.round()
    tp = np.sum((y_true == 1) & (y_pred_rounded == 1))
    tn = np.sum((y_true == 0) & (y_pred_rounded == 0))
    fp = np.sum((y_true == 0) & (y_pred_rounded == 1))
    fn = np.sum((y_true == 1) & (y_pred_rounded == 0))

    accuracy = (tp + tn) / (tp + tn + fp + fn)
    precision = tp / (tp + fp) if (tp + fp) > 0 else 0
    sensitivity = tp / (tp + fn) if (tp + fn) > 0 else 0
    specificity = tn / (tn + fp) if (tn + fp) > 0 else 0

    return accuracy, precision, sensitivity, specificity

# 10-Fold Cross-Validation
kf = KFold(n_splits=10, shuffle=True, random_state=42)
cross_val_metrics = []
fold = 1

for train_index, test_index in kf.split(X):
    print(f"\nFold {fold}:")
    fold += 1

    # Split data into train and test for this fold
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    # Build and train the model
    model = build_model(input_dim=X_train.shape[1])
    model, epoch_accuracies, epoch_losses = optimize_levenberg_marquardt(X_train, y_train, model)

    # Evaluate the model on the test set
    test_predictions = model.predict(X_test, verbose=0).flatten()
    accuracy, precision, sensitivity, specificity = calculate_metrics(y_test.values, test_predictions)

    cross_val_metrics.append((accuracy, precision, sensitivity, specificity))
    print(f"Accuracy: {accuracy:.4f}, Precision: {precision:.4f}, Sensitivity: {sensitivity:.4f}, Specificity: {specificity:.4f}")

# Print the average metrics across all folds
avg_metrics = np.mean(cross_val_metrics, axis=0)
print(f"\nAverage Metrics:\nAccuracy: {avg_metrics[0]:.4f}, Precision: {avg_metrics[1]:.4f}, Sensitivity: {avg_metrics[2]:.4f}, Specificity: {avg_metrics[3]:.4f}")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).

Fold 1:
`gtol` termination condition is satisfied.
Function evaluations 616, initial cost 4.3417e+01, final cost 3.8078e+01, first-order optimality 1.89e+01.
Accuracy: 0.8158, Precision: 0.0000, Sensitivity: 0.0000, Specificity: 0.9118

Fold 2:
`gtol` termination condition is satisfied.
Function evaluations 176, initial cost 3.5203e+01, final cost 3.3578e+01, first-order optimality 1.75e+01.
Accuracy: 0.6579, Precision: 0.0000, Sensitivity: 0.0000, Specificity: 1.0000

Fold 3:
`gtol` termination condition is satisfied.
Function evaluations 381, initial cost 2.6555e+01, final cost 2.6551e+01, first-order optimality 4.59e-01.
Accuracy: 0.7368, Precision: 0.0000, Sensitivity: 0.0000, Specificity: 1.0000

Fold 4:
`gtol` termination condition is satisfied.
Function evaluations 282, initial cost 5.1938e+01, final cost 4.7455e+01, first-order optimality 2.92e+01.
A